In [1]:
import pandas as pd
import numpy as np
import scipy
import csv

text = pd.read_csv("all_opinions.csv",encoding='ISO-8859-1')

issues = pd.read_csv("database.csv",encoding='ISO-8859-1')

privacyCaseSet = set([])
#for each row in the issues column of issues csv
issuesFile = open("database.csv")
issuesReader = csv.reader(issuesFile, delimiter=',')
for row in issuesReader:
    if row[39]:
    #if the integers value matches 30040, 50010, 50020, 50030, or 50040 (floats)
        privacyCaseSet.add(row[0])
        #add case_id value of same row to set
#print(privacyCaseSet)
print(len(privacyCaseSet))

8684


In [2]:
from sklearn import preprocessing
from sklearn.feature_extraction.text import CountVectorizer
import random
from sklearn.decomposition import LatentDirichletAllocation 

csv.field_size_limit(100000000)
opinionsFile = open("all_opinions.csv")
opinionsReader = csv.reader(opinionsFile, delimiter=',')
dictionary = {}
count = 0

for r in opinionsReader:   
    
    if r[9] in privacyCaseSet:
        #count += 1 
        if r[9]+'-01' in dictionary.keys():
            if r[9]+'-02' in dictionary.keys():
                if r[9]+'-03' in dictionary.keys():
                    if r[9]+'-04' in dictionary.keys():
                        dictionary[r[9]+'-05'] = r[-1]
                    else: dictionary[r[9]+'-04'] = r[-1]
                else: dictionary[r[9]+'-03'] = r[-1]
            else: dictionary[r[9]+'-02'] = r[-1]
        else: dictionary[r[9]+'-01'] = r[-1]     
            

print(len(dictionary))
print(count)

with open('dict2.csv', 'w') as csv_file:
    writer = csv.writer(csv_file)
   # writer.writerow(['id','text'])
    for key, value in dictionary.items():
        writer.writerow([key, value])
        
test = pd.read_csv('dict2.csv',error_bad_lines=False,encoding='ISO-8859-1',names=['id','text'])

14948
0


In [9]:
import nltk

testList = []

for entry in dictionary.values():
    testTokens = nltk.word_tokenize(entry)
    if "privacy" in testTokens:
        testList.append(entry)

print(len(testList))

838


In [12]:
num_features = 1000
tf_vectorizer = CountVectorizer(max_df=.2, min_df=.05, max_features=num_features, stop_words='english')

data_samples = testList

tf_data_samples = tf_vectorizer.fit_transform(data_samples) # Tokenizing and getting the feature counts. 
tf_feature_names = tf_vectorizer.get_feature_names()

In [13]:
num_topics = 15
lda = LatentDirichletAllocation(n_components=num_topics, max_iter=500, learning_method='online', learning_offset=50.,random_state=1).fit(tf_data_samples)

lda.score(tf_data_samples)

from IPython.display import display
def print_topics(model, vectorizer, top_n=8):    #### This is the code to print the topics. top_n can be changed. 
    for idx, topic in enumerate(model.components_):
        print("Topic %d:" % (idx))
        print([(vectorizer.get_feature_names()[i], topic[i])
                        for i in topic.argsort()[:-top_n - 1:-1]])

print_topics(lda,tf_vectorizer)

Topic 0:
[('ordinance', 727.7025162418108), ('election', 531.895834257017), ('commission', 381.2856354490053), ('commercial', 341.35215194130626), ('organization', 325.37039637626174), ('restriction', 267.7239199316647), ('appellants', 249.03711712503545), ('violence', 225.38228988831182)]
Topic 1:
[('religious', 755.9405506382493), ('surveillance', 656.011826084225), ('electronic', 504.13316079720755), ('conversations', 430.9634610781482), ('telephone', 414.1074493191385), ('katz', 336.41918521627093), ('title', 290.90960040273814), ('communications', 253.10872555167754)]
Topic 2:
[('vehicle', 736.4662688039926), ('stop', 656.9041976355785), ('terry', 597.0642036126378), ('ct', 480.45165843932836), ('traffic', 429.85985815380104), ('detention', 335.2403257802848), ('border', 296.75537178098506), ('automobile', 295.7114040743438)]
Topic 3:
[('cite', 515.6605453634814), ('____', 510.47115049990157), ('blood', 488.5249605513684), ('___', 443.4483559166269), ('alcohol', 321.223345869584),

In [ ]:
#explore fragment the data, optimize variables see what we get
#test our best results with word intrusion
#see which topics are different between time periods